## Proxy

| Field | Content |
| --- | --- |
| Class | Proxy |
| Collaborator | Method Request, Scheduler, Future |
| Responsibility | Defines the active object's interface to clients; Creates Method Requests; Runs in the client's thread |

## Method Request

| Field | Content |
| --- | --- |
| Class | Method Request |
| Collaborator | Servant, Future |
| Responsibility | Represents a method call on the active object; Provides guards to check when the method request becomes runnable |

## Concrete Method Request

| Field | Content |
| --- | --- |
| Class | Concrete Method Request |
| Collaborator | Servant, Future |
| Responsibility | Implements the representation of a specific method call; Implements guards |

## Active Object Collaboration Diagram

```mermaid
classDiagram
  direction LR

  class Client
  class Proxy {
    method_1()
    method_N()
  }
  class Scheduler {
    dispatch()
    insert()
  }
  class ActivationQueue {
    enqueue()
    dequeue()
  }
  class MethodRequest {
    can_run()
    call()
  }
  class ConcreteMethodRequest1
  class ConcreteMethodRequestN
  class Servant {
    method_1()
    method_N()
  }
  class Future

  Client ..> Proxy : invoke
  Proxy "1" -- "1" Scheduler : schedule
  Scheduler "1" -- "1" ActivationQueue : owns
  Proxy ..> Future : create
  Proxy ..> MethodRequest : create
  ActivationQueue "1" --> "*" MethodRequest : maintain
  MethodRequest --> Servant : execute
  MethodRequest --> Future : write to
  Client --> Future : obtain result from
  MethodRequest <|-- ConcreteMethodRequest1 : inherits
  MethodRequest <|-- ConcreteMethodRequestN : inherits
```


## Active Object Sequence Diagram

```mermaid
sequenceDiagram
  participant Client
  participant Proxy
  participant Future
  participant Scheduler
  participant ActivationList
  participant MethodRequest
  participant Servant

  Client->>Proxy: method()
  Proxy->>Future: create
  Proxy->>MethodRequest: create
  Proxy->>Scheduler: insert()
  Scheduler->>ActivationList: insert()
  Scheduler-->>Proxy: MethodRequest
  Proxy-->>Client: Future

  Scheduler->>ActivationList: dispatch()
  ActivationList-->>Scheduler: remove()
  Scheduler->>MethodRequest: can_run()
  MethodRequest->>Servant: call()
  Servant->>Servant: method()
  Servant-->>MethodRequest: Result
  MethodRequest-->>Future: write to future

  Client->>Future: read from future
  Future-->>Client: Result
```


Come si vede qui il client proxy (che gira in un thread) richiama un metodo sincronizzato. Questo metodo sincronizzato:
1. crea un `MethodRequest` (simile a un command)
2. chiama il metodo `insert()`

Il MethodRequest è una classe Facade che verrà poi specializzata dai ConcreteRequestMethod col solito giochetto puntatore padre = new puntatore figlio. Lo scheduler è in un altro thread, è parte del ActiveObject. Monitora col thread ed esegue. Esegue dispacciando la richiesta al servant. 

## Implementazione
Implementiamo un `ActiveObject` per sincronizzare un MQServant che farà semplicemente un'operazione lenta.

### Dipendenze

Poiché stiamo parlando di thread usiamo il supporto C++ al threading.

In [11]:
#include <iostream>
#include <string>
#include <thread>

using namespace std::chrono_literals;


In [12]:

static void log_line(const std::string& s) {
  std::cout << "[" << std::this_thread::get_id() << "] " << s << "\n";
}


In [13]:

  std::cout << "Hello, World of concurrency\n";


Hello, World of concurrency


In [14]:
  const std::string s = "This is a log message from the main thread";
  log_line(s);


[133606501812800] This is a log message from the main thread


Qui lancio in un altro thread e mi aspetto un TID diverso.

In [15]:

  const std::string s = "This is a log message from thread 1";
  std::thread t1(log_line, s);
  t1.join();



[133606342850240] This is a log message from thread 1


### Attore 1: `MQ_Servant`

Questo implementa le funzionalità core da sincronizzare. Qui dovremmo usare la solita tecnica corretta del C++ con la *Rule of 5* ma non lo farò... Andrò diretto al cuore della classe.
Ho 2 predicati che indicano 3 stati:
```mermaid
stateDiagram-v2
    [*] --> Empty

    Empty --> Neither: put() / not full()
    Neither --> Full: put() / becomes full
    Neither --> Neither: put() / still not full

    Full --> Neither: get() / not empty()
    Neither --> Empty: get() / becomes empty
    Neither --> Neither: get() / still not empty

    Empty --> Empty: get() blocked by empty()
    Full --> Full: put() blocked by full()

```

In [ ]:
class MQ_Servant{
    public:
        MQ_Servant(size_t mq_size) : mq_size_(mq_size) {}
        ~MQ_Servant() = default;
        
    // Message Queue implementation
    void put(const Message& msg);
    Message get();

    // Predicates
    bool is_full() const;
    bool is_empty() const;

    private:
        std::queue<Message> mq_;
        size_t mq_size_;

}